# 01_EDA v0.1: Análisis Exploratorio de Datos (Robusto)

Este notebook realiza un análisis exploratorio exhaustivo de los datos disponibles para el proyecto de detección de fraude financiero. 

**Objetivos de esta versión 0.1**:
1. Cargar y revisar todos los archivos de `data/raw/` (o `data/backup/` si los ligeros no existen en raw).  
2. Normalizar tipos (fechas, montos, flags “Yes/No” → 1/0) y revisar datos faltantes.  
3. Combinar transacciones con etiquetas de forma correcta (según la estructura real de `train_fraud_labels.json`).  
4. Detectar duplicados, outliers y valores inconsistentes.  
5. Explorar:
   - Distribución de montos (`amount`).  
   - Balance de la variable objetivo (`target`).  
   - Fraude por estado y ciudad.  
   - Fraude por hora del día y día de la semana.  
   - Fraude por tipo de tarjeta.  
   - Top MCCs donde hay más fraudes.  
6. Calcular algunas correlaciones básicas y visualizar mapa de calor.  
7. Hacer un pequeño preprocesamiento (generar columna `hour`, flags binarias, parsear fechas).  
8. Guardar el dataset combinado limpio en `data/processed/` para la fase de modelado.

In [22]:
# 2.1 Importar librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob

In [23]:
# 2.2 Ajustes de estilo para gráficos
%matplotlib inline
plt.style.use('default')
# 2.3 Opciones de pandas para mostrar más columnas/filas si es necesario
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

## 3.1 Carga de `transactions_data.csv`

Cargaremos el CSV completo de transacciones. Este archivo contiene todas las transacciones que serán nuestro principal objeto de estudio.


In [ ]:
# Ruta al CSV de transacciones
path_transactions = "../data/backup/transactions_data.csv"

# Cargar en un DataFrame
df_transacciones = pd.read_csv(path_transactions)

# Mostrar dimensiones y algunas filas
print("Dimensiones de df_transacciones:", df_transacciones.shape)
df_transacciones.head(5)


Dimensiones de df_transacciones: (13305915, 12)


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.00,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.00,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.00,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.00,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.00,5813,NaN


## 3.2 Carga de `train_fraud_labels.json`

Este archivo JSON contiene las etiquetas (isFraud o campo equivalente) para identificar si cada transacción es fraudulenta o no. Lo convertiremos en DataFrame para su posterior unión.


In [24]:
# Ruta al JSON de etiquetas
path_labels = "../data/backup/train_fraud_labels.json"

# Cargar JSON y convertir a DataFrame
with open(path_labels, "r") as f:
    data_labels = json.load(f)
df_labels = pd.DataFrame(data_labels)

# Mostrar dimensiones y primeras filas
print("Dimensiones de df_labels:", df_labels.shape)
df_labels.head(5)


Dimensiones de df_labels: (8914963, 1)


,target
10649266,No
23410063,No
9316588,No
12478022,No
9558530,No


## 3.3 Carga de datasets ligeros

- **cards_data.csv**: Información de las tarjetas (posiblemente con features como tipo de tarjeta, país, etc.).  
- **users_data.csv**: Datos de usuarios (features demográficos, ubicación, etc.).  
- **mcc_codes.json**: Diccionario de códigos MCC y sus descripciones (Merchant Category Codes).


In [16]:
import pandas as pd
import json
import os

# 1) Verificar existencia de los archivos en data/backup/
print("cards_data.csv existe en backup? ",  os.path.exists("../data/backup/cards_data.csv"))
print("users_data.csv existe en backup?  ",  os.path.exists("../data/backup/users_data.csv"))
print("mcc_codes.json existe en backup? ", os.path.exists("../data/backup/mcc_codes.json"))

# 2) Definir rutas
path_cards = "../data/backup/cards_data.csv"
path_users = "../data/backup/users_data.csv"
path_mcc   = "../data/backup/mcc_codes.json"

# 3) Cargar cada uno
df_cards = pd.read_csv(path_cards)
df_users = pd.read_csv(path_users)

# Cargar mcc_codes.json y convertir a DataFrame de dos columnas (código y descripción)
with open(path_mcc, "r") as f:
    mcc_dict = json.load(f)

# Supongamos que mcc_dict = { "5411": "Grocery Stores", "5812": "Eating Places/Restaurants", ... }
# Queremos un DataFrame con columnas ["mcc_code", "description"].
df_mcc = pd.DataFrame(
    list(mcc_dict.items()),
    columns=["mcc_code", "description"]
)

# 4) Mostrar información básica
print("Dimensiones de df_cards:", df_cards.shape)
display(df_cards.head())

print("\nDimensiones de df_users:", df_users.shape)
display(df_users.head())

print("\nDimensiones de df_mcc:", df_mcc.shape)
display(df_mcc.head())



cards_data.csv existe en backup?  True
users_data.csv existe en backup?   True
mcc_codes.json existe en backup?  True
Dimensiones de df_cards: (6146, 13)


,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No



Dimensiones de df_users: (2000, 14)


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1



Dimensiones de df_mcc: (109, 2)


,mcc_code,description
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


In [ ]:
## 4.1 Inspección de `df_transacciones` y `df_labels`

Revisaremos para cada DataFrame:
1. Dimensiones (`shape`)  
2. Tipos de cada columna (`dtypes`)  
3. Conteo de valores faltantes  
4. Filas duplicadas (exactas) y duplicados por ID (si aplica)


In [17]:
# 4.1 Función auxiliar para inspeccionar un DataFrame
def inspeccionar_df(df, nombre):
    print(f"--- Inspección de {nombre} ---")
    print("Dimensiones:", df.shape)
    print("\nTipos de columnas:\n", df.dtypes)
    print("\nValores faltantes por columna:\n", df.isnull().sum().sort_values(ascending=False))
    print("\nDuplicados exactos (filas idénticas):", df.duplicated().sum())
    if "transaction_id" in df.columns:
        print("Duplicados en 'transaction_id':", df["transaction_id"].duplicated().sum())
    print("-" * 60 + "\n")

# 4.1.1 Inspeccionar df_transacciones
inspeccionar_df(df_transacciones, "df_transacciones")

# 4.1.2 Inspeccionar df_labels
inspeccionar_df(df_labels, "df_labels")


--- Inspección de df_transacciones ---
Dimensiones: (13305915, 12)

Tipos de columnas:
 id                  int64
date               object
client_id           int64
card_id             int64
amount             object
use_chip           object
merchant_id         int64
merchant_city      object
merchant_state     object
zip               float64
mcc                 int64
errors             object
dtype: object

Valores faltantes por columna:
 errors            13094522
zip                1652706
merchant_state     1563700
id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
mcc                      0
dtype: int64

Duplicados exactos (filas idénticas): 0
------------------------------------------------------------

--- Inspección de df_labels ---
Dimensiones: (8914963, 1)

Tipos de columnas:
 target    object
dtype: object

Va

In [ ]:
## 4.2 Inspección de `df_cards`, `df_users` y `df_mcc`

Revisaremos para cada uno:
1. Dimensiones  
2. Tipos de columnas  
3. Valores faltantes  
4. Duplicados exactos (si aplican)


In [18]:
# 4.2.1 Inspeccionar df_cards
inspeccionar_df(df_cards, "df_cards")

# 4.2.2 Inspeccionar df_users
inspeccionar_df(df_users, "df_users")

# 4.2.3 Inspeccionar df_mcc
inspeccionar_df(df_mcc, "df_mcc")


--- Inspección de df_cards ---
Dimensiones: (6146, 13)

Tipos de columnas:
 id                        int64
client_id                 int64
card_brand               object
card_type                object
card_number               int64
expires                  object
cvv                       int64
has_chip                 object
num_cards_issued          int64
credit_limit             object
acct_open_date           object
year_pin_last_changed     int64
card_on_dark_web         object
dtype: object

Valores faltantes por columna:
 id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64

Duplicados exactos (filas idénticas): 0
--------------------------------------------------------

## 5.1 Merge de `df_transacciones` con `df_labels`

Realizamos un `left join` usando `transaction_id` como clave para conservar todas las transacciones y agregar la etiqueta (`isFraud`) correspondiente.


In [20]:
# Verificar formas
print("Filas en df_transacciones:", df_transacciones.shape[0])
print("Filas en df_labels:", df_labels.shape[0])

# Opcional: mostrar los primeros 5 valores de df_labels para ver si la columna 'target' existe
print("\nPrimeras 5 filas de df_labels:")
display(df_labels.head(5))


Filas en df_transacciones: 13305915
Filas en df_labels: 8914963

Primeras 5 filas de df_labels:


,target
10649266,No
23410063,No
9316588,No
12478022,No
9558530,No
